In [19]:
import requests

In [20]:
class InvestmentPortfolio:
    def __init__(self, tickers, shares):
        self.tickers = tickers
        self.number_of_shares = shares


In [21]:
def printResults(s):
    if s is not None:
        print("Stocks suggested for you: ", s.tickers)
        print("Corresponding shares for each stock: ", s.number_of_shares)
    else:
        print("Please try again.")

In [22]:
import datetime

In [70]:
def get_current_price(date, stock_symbol):
    req = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=' + stock_symbol + '&apikey=Z552XY642X21K4LB')
    close = 1000
    if req is not None:
        #print(req.json())
        count = 0
        #print('tests', 'Note' in req.json())
        while 'Note' in req.json() and count<10000:
            count+=1
            if(count%100==0):
                req = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=' + stock_symbol + '&apikey=Z552XY642X21K4LB')    
        if('Note' not in req.json()):
            close =  req.json()['Time Series (Daily)'][date]['4. close'] 
        print('count', count)
        if count>=10000:
            print("API access error, please try again later")
    else:
        print("API access error, please try again later")
        
    return close

In [71]:
def get_monthly_low_high(date, stock_symbol):
#    print(datetime.datetime.now())
    req = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=' + stock_symbol + '&apikey=Z552XY642X21K4LB')
#     r.status_code
#     r.text
    #print(date)
    low=0
    high=0
    if req is not None:
        count = 0
        #print('tests', 'Note' in req.json())
        while 'Note' in req.json() and count<5000:
            count+=1
            if(count%100==0):
                req = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=' + stock_symbol + '&apikey=Z552XY642X21K4LB')
        if('Note' not in req.json()):
            low = req.json()['Monthly Time Series'][date]['3. low']
            high = req.json()['Monthly Time Series'][date]['2. high']
        #print('count', count) 
        if count>=5000:
            print("API access error, please try again later")
    else:
        print("API access error, please try again later")
    #print(low, high)

    return low, high

In [72]:
def distribute(money, stocks_list):
    date = '2018-12-14'
    #stock_name = 'MSFT'
    percent = []
    for stock_name in stocks_list:
        print(stock_name)
        low = 100
        high = 200
        low_s, high_s = get_monthly_low_high(date, stock_name)
        low = float(low_s)
        high = float(high_s)
        current = float(get_current_price(date,stock_name))
        print('l,h', low, high)
        print('c', current)
        if current<low and low!=0:
            percent.append([(low-current)/low+1.00,stock_name, current])
        elif current > high:
            print('price too high')
            percent.append([0.01, stock_name, current])
        else:
            print('l,h,c', low, high, current)
            range_ = high-low
            if range_== 0:
                range_ = 1
            loc = (current-low)/range_
            percent.append([-1* loc +1.00, stock_name, current]) 
    print('before', percent)

    #percent.sort( reverse=True)
    percent.sort()
    
    print('after',percent)
    
    sum_= 0.00
    for p in percent:
        print('p',p[0])
        sum_+= p[0]
    print('sum', sum_)
    if sum_ <=0:
        sum_ = 1
    for p in percent:
        p[0]= p[0]/sum_
    print('new p', percent)
       
    for p in percent:
        
        print('money, price',money, p[2])
        share = int(money*p[0]/p[2])
        print('money, price, share',money, p[2], share)

        
        #money = money - share*p[2]
        p.append(share)
    
    print ('final', percent)
    
    val =0
    for p in percent:
        val+=p[2]*p[3]
    print('val', val)
    return percent

In [73]:
# Due to the limitation of 5 API calls per minute from the free API we use, if you put two strategies, 
# there will be 6 stocks (greater than 5 API calls per minute) 
# therefore you need to wait a little while for the program to finish
distribute(50000, ['AAPL','GOOG'])
#distribute(50000, ['MSFT','TWLO','GOOG','A','FB','AMZN'])

AAPL
count 0
l,h 163.33 184.94
c 165.48
l,h,c 163.33 184.94 165.48
GOOG
count 0
l,h 1023.29 1124.65
c 1042.1
l,h,c 1023.29 1124.65 1042.1
before [[0.900509023600186, 'AAPL', 165.48], [0.8144238358326764, 'GOOG', 1042.1]]
after [[0.8144238358326764, 'GOOG', 1042.1], [0.900509023600186, 'AAPL', 165.48]]
p 0.8144238358326764
p 0.900509023600186
sum 1.7149328594328623
new p [[0.47490129502913064, 'GOOG', 1042.1], [0.5250987049708694, 'AAPL', 165.48]]
money, price 50000 1042.1
money, price, share 50000 1042.1 22
money, price 50000 165.48
money, price, share 50000 165.48 158
final [[0.47490129502913064, 'GOOG', 1042.1, 22], [0.5250987049708694, 'AAPL', 165.48, 158]]
val 49072.03999999999


[[0.47490129502913064, 'GOOG', 1042.1, 22],
 [0.5250987049708694, 'AAPL', 165.48, 158]]

In [74]:
def get_recommendations(strategy):
    dic=[]
    
    if strategy == 'ethical':  
        print(1)
        dic.append('AAPL')
        dic.append('ADBE')
        dic.append('VOO')
    elif strategy == 'growth':  
        print(2)  
        dic.append('JPM')
        dic.append('BAC')
        dic.append('WFC')
    elif strategy == 'index':
        print(3)
        dic.append('GRPN')
        dic.append('WMT')
        dic.append('AMD')
    elif strategy == 'quality':
        print(4)
        dic.append('GOOGL')
        dic.append('AMZN')
        dic.append('MSFT')

    elif strategy == 'value':
        print(5)
        dic.append('TWLO')
        dic.append('NVDA')
        dic.append('MDB')
    
    return dic    

In [75]:
#extra feature: mode = aggressive, balanced, conventional
#to be determined how it works

In [76]:
def input_validation(lower_case_strategies, investment_amount, strategy1, strategy2=None):
    #input validation
    
    #Minimum is $5000 USD
    if(investment_amount<5000.00):
        error = "Error: Minimum investment amount is $5000 USD. You entered: $"+str(investment_amount)+" USD."
        print(error)
        return False, []
    strategy_list = [] #lower cased strategies
    if strategy1 is not None:
        lower_case_strategy1 = strategy1.lower()
        strategy_list.append(lower_case_strategy1)
    if strategy2 is not None:
        lower_case_strategy2 = strategy2.lower()
        strategy_list.append(lower_case_strategy2)
    error1 = "Error: Strategy must be one of the following: "+ str(lower_case_strategies) 
    error2 = "You entered: "
    flag = False
    if strategy1 is not None and lower_case_strategy1 not in lower_case_strategies:
        error2 += '\''+ strategy1 + '\''
        flag = True
    if strategy2 is not None and lower_case_strategy2 not in lower_case_strategies:
        if flag:
            error2 += ' and '
        error2 += '\''+strategy2 + '\''
        flag = True
    if flag:
        print(error1)
        print(error2)
    return flag, strategy_list

In [77]:
def getStocks(investment_amount=5000.00, strategy1='Ethical', strategy2=None):
    
    
    #Strategies need to be 'Ethical', 'Growth', 'Index', 'Quality' or 'Value'
    strategies = ['Ethical', 'Growth', 'Index', 'Quality', 'Value' ]
    lower_case_strategies = [s.lower() for s in strategies]
    
    
     #input validation
    invalid, strategy_list = input_validation(lower_case_strategies, investment_amount, strategy1, strategy2)
    if invalid or not strategy_list:
        return
        
    
    
    
    shares=[]
    #tickers = ['TICK', 'SYM', 'BOLS']
    #shares = [10, 20, 30]
    stocks= []
    for strategy in strategy_list:
        stocks.extend(get_recommendations(strategy))
        print(stocks)
        
   
    results = distribute(investment_amount, stocks)
    print('here', results);        
    
    #return(InvestmentPortfolio(stocks, shares))
    return results


In [78]:
s1 = getStocks(80000, 'index')
s1

3
['GRPN', 'WMT', 'AMD']
GRPN
count 4900
l,h 2.98 3.33
c 3.14
l,h,c 2.98 3.33 3.14
WMT
count 0
l,h 91.57 99.59
c 91.85
l,h,c 91.57 99.59 91.85
AMD
count 0
l,h 19.17 23.75
c 19.9
l,h,c 19.17 23.75 19.9
before [[0.5428571428571426, 'GRPN', 3.14], [0.9650872817955112, 'WMT', 91.85], [0.840611353711791, 'AMD', 19.9]]
after [[0.5428571428571426, 'GRPN', 3.14], [0.840611353711791, 'AMD', 19.9], [0.9650872817955112, 'WMT', 91.85]]
p 0.5428571428571426
p 0.840611353711791
p 0.9650872817955112
sum 2.348555778364445
new p [[0.23114509259609456, 'GRPN', 3.14], [0.3579269274571797, 'AMD', 19.9], [0.4109279799467256, 'WMT', 91.85]]
money, price 80000 3.14
money, price, share 80000 3.14 5889
money, price 80000 19.9
money, price, share 80000 19.9 1438
money, price 80000 91.85
money, price, share 80000 91.85 357
final [[0.23114509259609456, 'GRPN', 3.14, 5889], [0.3579269274571797, 'AMD', 19.9, 1438], [0.4109279799467256, 'WMT', 91.85, 357]]
val 79898.10999999999
here [[0.23114509259609456, 'GRPN', 3.

[[0.23114509259609456, 'GRPN', 3.14, 5889],
 [0.3579269274571797, 'AMD', 19.9, 1438],
 [0.4109279799467256, 'WMT', 91.85, 357]]

In [69]:
s2 = getStocks(10000, 'ethical', 'growth')
s2

1
['AAPL', 'ADBE', 'VOO']
2
['AAPL', 'ADBE', 'VOO', 'JPM', 'BAC', 'WFC']
AAPL
count 0
l,h 163.33 184.94
c 165.48
l,h,c 163.33 184.94 165.48
ADBE
count 0
l,h 229.91 260.72
c 230.0
l,h,c 229.91 260.72 230.0
VOO
count 0
l,h 237.62 257.74
c 239.38
l,h,c 237.62 257.74 239.38
JPM
count 5000
API access error, please try again later
l,h 99.28 112.89
c 100000.0
price too high
BAC
API access error, please try again later
count 3800
l,h 0.0 0.0
c 24.48
price too high
WFC
count 0
l,h 46.3834 55.04
c 46.54
l,h,c 46.3834 55.04 46.54
before [[0.900509023600186, 'AAPL', 165.48], [0.9970788704965919, 'ADBE', 230.0], [0.9125248508946326, 'VOO', 239.38], [0.01, 'JPM', 100000.0], [0.01, 'BAC', 24.48], [0.9819097567174182, 'WFC', 46.54]]
after [[0.01, 'BAC', 24.48], [0.01, 'JPM', 100000.0], [0.900509023600186, 'AAPL', 165.48], [0.9125248508946326, 'VOO', 239.38], [0.9819097567174182, 'WFC', 46.54], [0.9970788704965919, 'ADBE', 230.0]]
p 0.01
p 0.01
p 0.900509023600186
p 0.9125248508946326
p 0.9819097567174

[[0.002623279373486716, 'BAC', 24.48, 0],
 [0.002623279373486716, 'JPM', 100000.0, 0],
 [0.23622867472490305, 'AAPL', 165.48, 11],
 [0.2393807619145931, 'VOO', 239.38, 8],
 [0.2575823611422163, 'WFC', 46.54, 44],
 [0.2615616434713142, 'ADBE', 230.0, 9]]

In [60]:
s3 = getStocks(80000, 'quality','value')
s3

4
['GOOGL', 'AMZN', 'MSFT']
5
['GOOGL', 'AMZN', 'MSFT', 'TWLO', 'NVDA', 'MDB']
GOOGL
count 0
l,h 1033.0 1135.0
c 1051.71
l,h,c 1033.0 1135.0 1051.71
AMZN
count 0
l,h 1585.0 1778.34
c 1591.91
l,h,c 1585.0 1778.34 1591.91
MSFT
count 300
l,h 103.89 113.42
c 106.03
l,h,c 103.89 113.42 106.03
TWLO
count 0
l,h 85.6814 100.47
c 91.62
l,h,c 85.6814 100.47 91.62
NVDA
count 0
l,h 144.82 174.68
c 146.45
l,h,c 144.82 174.68 146.45
MDB
count 0
l,h 79.13 93.23
c 90.58
l,h,c 79.13 93.23 90.58
before [[0.81656862745098, 'GOOGL', 1051.71], [0.964259853108513, 'AMZN', 1591.91], [0.7754459601259182, 'MSFT', 106.03], [0.5984339288370768, 'TWLO', 91.62], [0.9454119223040859, 'NVDA', 146.45], [0.18794326241134784, 'MDB', 90.58]]
after [[0.18794326241134784, 'MDB', 90.58], [0.5984339288370768, 'TWLO', 91.62], [0.7754459601259182, 'MSFT', 106.03], [0.81656862745098, 'GOOGL', 1051.71], [0.9454119223040859, 'NVDA', 146.45], [0.964259853108513, 'AMZN', 1591.91]]
p 0.18794326241134784
p 0.5984339288370768
p 0.775

[[0.04382940225445172, 'MDB', 90.58, 38],
 [0.13955808286601548, 'TWLO', 91.62, 121],
 [0.18083826191417793, 'MSFT', 106.03, 136],
 [0.19042829405920528, 'GOOGL', 1051.71, 14],
 [0.22047525890089217, 'NVDA', 146.45, 120],
 [0.22487070000525725, 'AMZN', 1591.91, 11]]

In [61]:
#some test cases below

In [62]:
s = getStocks(50, 'ethic', "ethical")
printResults(s)

Error: Minimum investment amount is $5000 USD. You entered: $50 USD.
Please try again.


In [63]:
s = getStocks(80000, 'hahaha', "ethical")
printResults(s)

Error: Strategy must be one of the following: ['ethical', 'growth', 'index', 'quality', 'value']
You entered: 'hahaha'
Please try again.


In [64]:
s = getStocks(80000, 'ethical', "hohoho")
printResults(s)

Error: Strategy must be one of the following: ['ethical', 'growth', 'index', 'quality', 'value']
You entered: 'hohoho'
Please try again.


In [65]:
s = getStocks(80000, 'hahaha', "hohoho")
printResults(s)

Error: Strategy must be one of the following: ['ethical', 'growth', 'index', 'quality', 'value']
You entered: 'hahaha' and 'hohoho'
Please try again.


In [66]:
s = getStocks(80000, 'ethic')
printResults(s)

Error: Strategy must be one of the following: ['ethical', 'growth', 'index', 'quality', 'value']
You entered: 'ethic'
Please try again.
